# Idea is to take the BED files with edits and run featureCounts on them, but featureCounts needs a BAM file as input.
- This notebook was formerly: La_calculate_epkm_sc.ipynb (/home/bay001/projects/ryan_editing_20190314/permanent_data/43_new_scRNA_editing/La_calculate_epkm_sc.ipynb)
- This notebook takes edits (merged from both strands) from each bin (since there are 20k cells to account for, I've split each run based on the first 2 bases of each barcode) and runs featureCounts. Then it joins all outputs together to form one dataframe.
- This notebook also counts edits across just the 3'UTR for comparison to bulk.

In [1]:
%matplotlib inline

import glob
import os
import pandas as pd
import gffutils
import pysam
import pybedtools
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from collections import OrderedDict
from tqdm import tnrange, tqdm_notebook
from qtools import Submitter

# Important! Filter the edit file for conf score

In [2]:
conf=0.9

In [3]:
input_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/inputs/'
output_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/outputs/'

sailor_output_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/sailor_outputs_individual_barcodes_merged_bedfiles'
tmp_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/tmp/for_featurecounts{}'.format(conf)

In [4]:
for nt in ['A', 'C', 'G', 'T']:
    for nt2 in ['A', 'C', 'G', 'T']:
        split_tmp_dir = os.path.join(tmp_dir, '{}{}'.format(nt, nt2))
        if not os.path.exists(split_tmp_dir):
            ! mkdir $split_tmp_dir

# Convert BED to BAM
- every 'alignment' is an edit site. 

In [6]:
all_bed_files = sorted(glob.glob(os.path.join(sailor_output_dir, '*-1.bed')))
print(len(all_bed_files))
all_bed_files[:3] # just like to see what the filenames look like so we can better parse them out in the cells below.

19611


['/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/sailor_outputs_individual_barcodes_merged_bedfiles/Apo_Control_possorted_genome_bam_MD-AAACCCAAGCCAGTAG-1.bed',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/sailor_outputs_individual_barcodes_merged_bedfiles/Apo_Control_possorted_genome_bam_MD-AAACCCAAGGATGCGT-1.bed',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/sailor_outputs_individual_barcodes_merged_bedfiles/Apo_Control_possorted_genome_bam_MD-AAACCCACAATACAGA-1.bed']

# Basically filter for conf, perform bedToBam and samtools sort on these guys.

In [7]:
def filter_bed(input_bed, output_bed, conf):
    """
    Filters the BED file 
    """
    edit_head = ['chrom','start','end','conf','frac','strand']
    df = pd.read_table(input_bed, names=edit_head)
    df = df[df['conf']>=conf]
    df.to_csv(output_bed, sep='\t', header=False, index=False)


genome = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/inputs/refdata-cellranger-hg19_lenti_common-3.0.0/star/chrNameLength.txt'
progress = tnrange(len(all_bed_files))
for bed in all_bed_files:
    prefix = os.path.basename(bed).split('possorted_genome_bam_MD-')[1][:2]                               # ie. possorted_genome_bam_MD-AAACCCAAGAGCCCAA-1.bed -> AA
    split_tmp_dir = os.path.join(tmp_dir, "{}".format(prefix))                                            # ie. AA -> for_featurecounts0.9/AA
    filter_fn = os.path.join(split_tmp_dir, os.path.basename(bed) + ".{}.bed".format(conf))               # ie. for_featurecounts0.9/AA/possorted_genome_bam_MD-AAACCCAAGAGCCCAA-1.bed.0.9.bed
    output_bam = os.path.join(split_tmp_dir, os.path.basename(bed).replace('.bed','.bam'))                # ie. for_featurecounts0.9/AA/possorted_genome_bam_MD-AAACCCAAGAGCCCAA-1.bam
    output_sorted_bam = os.path.join(split_tmp_dir, os.path.basename(bed).replace('.bed','.sorted.bam'))  # ie. for_featurecounts0.9/AA/possorted_genome_bam_MD-AAACCCAAGAGCCCAA-1.sorted.bam

    if not os.path.exists(output_sorted_bam):
        filter_bed(bed, filter_fn, conf)
        cmd = 'bedToBam '
        cmd += '-i {} '.format(filter_fn)
        cmd += '-g {} '.format(genome)
        cmd += '> {}'.format(output_bam)
        ! $cmd
        sort_cmd = 'samtools sort {} > {}'.format(output_bam, output_sorted_bam)
        ! $sort_cmd
    progress.update(1)

/projects/ps-yeolab4/bay001/anaconda2/envs/python2-essential/lib/python2.7/site-packages/ipykernel_launcher.py:6: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


# Run featureCounts to get the number of edits assigned to each gene.
- Run featureCounts 16 times, once in each "bin" to save resources

In [8]:
gtf = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/inputs/refdata-cellranger-hg19_lenti_common-3.0.0/genes/genes.gtf'

In [9]:
cmds = []
for nt in ['A', 'C', 'G', 'T']:
    for nt2 in ['A', 'C', 'G', 'T']:
        split_tmp_dir = os.path.join(tmp_dir, '{}{}'.format(nt, nt2))

        cmd = 'module load subreadfeaturecounts;featureCounts '
        cmd += '-a {} '.format(gtf)
        cmd += '-s 1 '
        cmd += '-o {}'.format(os.path.join(split_tmp_dir, 'counts_at_conf_{}.txt '.format(conf)))
        cmd += '-R CORE '
        cmd += os.path.join(split_tmp_dir, '*.sorted.bam')

        cmds.append(cmd)

In [10]:
print(len(cmds))
cmds[:2]

16


['module load subreadfeaturecounts;featureCounts -a /home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/inputs/refdata-cellranger-hg19_lenti_common-3.0.0/genes/genes.gtf -s 1 -o /home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/tmp/for_featurecounts0.9/AA/counts_at_conf_0.9.txt -R CORE /home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/tmp/for_featurecounts0.9/AA/*.sorted.bam',
 'module load subreadfeaturecounts;featureCounts -a /home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/inputs/refdata-cellranger-hg19_lenti_common-3.0.0/genes/genes.gtf -s 1 -o /home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/tmp/for_featurecounts0.9/AC/counts_at_conf_0.9.txt -R CORE /home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/tmp/for_featurecounts0.9/AC/*.sorted.bam']

In [11]:
Submitter(
    commands=cmds,
    job_name='featureCounts',
    sh='featureCounts.sh',
    array=True,
    nodes=1,
    ppn=2,
    walltime='48:00:00',
    submit=True,
)

Writing 16 tasks as an array-job.
Wrote commands to featureCounts.sh.
Submitted script to queue home.
 Job ID: 21965831


# Let's run featureCounts across just 3utrs also, in addition to the whole gene.
- Using the 10X ensembl-style three_prime_utr annotations, generate a SAF file and just call featureCounts on that.
- Run featureCounts 16 times, once in each "bin" to save resources

In [12]:
gtf_3utr_file = os.path.join(input_dir, 'genes.three_prime_utr_only.gtf')

! grep --color 'three_prime_utr' $gtf > $gtf_3utr_file

In [13]:
gtf_3utr = pd.read_csv(gtf_3utr_file, names=['chrom','src','region','start','end','.','strand','.','attr'], sep='\t')
gtf_3utr['geneid'] = gtf_3utr['attr'].str.extract("gene_id \"([\w\d\.]+)\"")
gtf_3utr = gtf_3utr[['geneid','chrom','start','end','strand']]
gtf_3utr.head()

/home/bay001/anaconda2/envs/python2-essential/lib/python2.7/site-packages/pandas/io/parsers.py:702: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


,geneid,chrom,start,end,strand
0,ENSG00000237683,1,137621,138529,-
1,ENSG00000237683,1,134901,135802,-
2,ENSG00000235249,1,368598,368634,+
3,ENSG00000185097,1,621059,621095,-
4,ENSG00000187634,1,879534,879955,+


In [14]:
gtf_3utr.to_csv(
    os.path.join(input_dir, 'genes.three_prime_utr_only.SAF'),
    sep='\t',
    index=False,
    header=False
)

In [15]:
saf = os.path.join(input_dir, 'genes.three_prime_utr_only.SAF')

In [16]:
cmds = []
for nt in ['A', 'C', 'G', 'T']:
    for nt2 in ['A', 'C', 'G', 'T']:
        split_tmp_dir = os.path.join(tmp_dir, '{}{}'.format(nt, nt2))

        cmd = 'module load subreadfeaturecounts;featureCounts '
        cmd += '-a {} '.format(saf)
        cmd += '-F SAF '
        cmd += '-s 1 '
        cmd += '-o {}'.format(os.path.join(split_tmp_dir, 'counts_at_conf_{}.three_prime_utr.txt '.format(conf)))
        cmd += '-R CORE '
        cmd += os.path.join(split_tmp_dir, '*.sorted.bam')

        cmds.append(cmd)

In [17]:
# Just check the command to make sure it's OK
cmds[0]

'module load subreadfeaturecounts;featureCounts -a /home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/inputs/genes.three_prime_utr_only.SAF -F SAF -s 1 -o /home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/tmp/for_featurecounts0.9/AA/counts_at_conf_0.9.three_prime_utr.txt -R CORE /home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/tmp/for_featurecounts0.9/AA/*.sorted.bam'

In [18]:
Submitter(
    commands=cmds,
    job_name='featureCounts-3utr',
    sh='featureCounts-3utr.sh',
    array=True,
    nodes=1,
    ppn=2,
    walltime='48:00:00',
    submit=True,
)

Writing 16 tasks as an array-job.
Wrote commands to featureCounts-3utr.sh.
Submitted script to queue home.
 Job ID: 21965832


# Read in featureCounts outputs from each bin and merge them now.
- each column is something like: /home/bay001/projects/ryan_editing_20190314/permanent_data/43_new_scRNA_editing/tmp/AG/possorted_genome_bam_MD-AGTGTTGTCGAACCAT-1.merged.sorted.bam
- original 'split-bam' files look like this: /home/bay001/projects/ryan_editing_20190314/permanent_data/43_new_scRNA_editing/bam_files/possorted_genome_bam_MD-AGTGTTGTCGAACCAT-1.sorted.bam

In [19]:
print("checking {}".format(os.path.join(tmp_dir, "*", "counts_at_conf_{}.txt".format(conf))))
all_counts = glob.glob(os.path.join(tmp_dir, "*", "counts_at_conf_{}.txt".format(conf)))
print(len(all_counts))
all_counts[:3]

checking /home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/tmp/for_featurecounts0.9/*/counts_at_conf_0.9.txt
16


['/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/tmp/for_featurecounts0.9/TG/counts_at_conf_0.9.txt',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/tmp/for_featurecounts0.9/CG/counts_at_conf_0.9.txt',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/tmp/for_featurecounts0.9/CT/counts_at_conf_0.9.txt']

In [20]:
merged = pd.DataFrame(index=[])
merged

""


In [21]:
progress = tnrange(16)
for nt in ['A', 'C', 'G', 'T']:
    for nt2 in ['A', 'C', 'G', 'T']:
        split_input_dir = os.path.join(tmp_dir, '{}{}'.format(nt, nt2))
        df = pd.read_csv(os.path.join(split_input_dir, 'counts_at_conf_{}.txt'.format(conf)), sep='\t', index_col=0, skiprows=1)
        df = df.iloc[:,5:]
        df.columns = [
            c.replace(split_input_dir, '') for c in df.columns
        ]
        merged = pd.merge(merged, df, how='outer', left_index=True, right_index=True)
        print("dataframe size: {}, NEW merged size: {}".format(df.shape, merged.shape))
        progress.update(1)
        

dataframe size: (32739, 997), NEW merged size: (32739, 997)
dataframe size: (32739, 1059), NEW merged size: (32739, 2056)
dataframe size: (32739, 1423), NEW merged size: (32739, 3479)
dataframe size: (32739, 1205), NEW merged size: (32739, 4684)
dataframe size: (32739, 1479), NEW merged size: (32739, 6163)
dataframe size: (32739, 962), NEW merged size: (32739, 7125)
dataframe size: (32739, 921), NEW merged size: (32739, 8046)
dataframe size: (32739, 1384), NEW merged size: (32739, 9430)
dataframe size: (32739, 1306), NEW merged size: (32739, 10736)
dataframe size: (32739, 780), NEW merged size: (32739, 11516)
dataframe size: (32739, 1116), NEW merged size: (32739, 12632)
dataframe size: (32739, 1469), NEW merged size: (32739, 14101)
dataframe size: (32739, 1063), NEW merged size: (32739, 15164)
dataframe size: (32739, 1668), NEW merged size: (32739, 16832)
dataframe size: (32739, 1421), NEW merged size: (32739, 18253)
dataframe size: (32739, 1358), NEW merged size: (32739, 19611)


In [22]:
merged.to_csv(os.path.join(output_dir, 'all_scRNA_barcodes_counts.conf{}.txt'.format(conf)), sep='\t')

# Read in featureCounts (3'utr) output

In [23]:
print("checking {}".format(os.path.join(tmp_dir, "*", "counts_at_conf_{}.three_prime_utr.txt".format(conf))))

all_counts = glob.glob(os.path.join(tmp_dir, "*", "counts_at_conf_{}.three_prime_utr.txt".format(conf)))
print(len(all_counts))
all_counts[:3]

checking /home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/tmp/for_featurecounts0.9/*/counts_at_conf_0.9.three_prime_utr.txt
16


['/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/tmp/for_featurecounts0.9/TG/counts_at_conf_0.9.three_prime_utr.txt',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/tmp/for_featurecounts0.9/CG/counts_at_conf_0.9.three_prime_utr.txt',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/tmp/for_featurecounts0.9/CT/counts_at_conf_0.9.three_prime_utr.txt']

In [24]:
merged = pd.DataFrame(index=[])
merged

""


In [25]:
progress = tnrange(16)
for nt in ['A', 'C', 'G', 'T']:
    for nt2 in ['A', 'C', 'G', 'T']:
        split_input_dir = os.path.join(tmp_dir, '{}{}'.format(nt, nt2))
        df = pd.read_csv(os.path.join(split_input_dir, 'counts_at_conf_{}.three_prime_utr.txt'.format(conf)), sep='\t', index_col=0, skiprows=1)
        df = df.iloc[:,5:]
        df.columns = [
            c.replace(split_input_dir, '') for c in df.columns
        ]
        merged = pd.merge(merged, df, how='outer', left_index=True, right_index=True)
        print("dataframe size: {}, NEW merged size: {}".format(df.shape, merged.shape))
        progress.update(1)
        

dataframe size: (19615, 997), NEW merged size: (19615, 997)
dataframe size: (19615, 1059), NEW merged size: (19615, 2056)
dataframe size: (19615, 1423), NEW merged size: (19615, 3479)
dataframe size: (19615, 1205), NEW merged size: (19615, 4684)
dataframe size: (19615, 1479), NEW merged size: (19615, 6163)
dataframe size: (19615, 962), NEW merged size: (19615, 7125)
dataframe size: (19615, 921), NEW merged size: (19615, 8046)
dataframe size: (19615, 1384), NEW merged size: (19615, 9430)
dataframe size: (19615, 1306), NEW merged size: (19615, 10736)
dataframe size: (19615, 780), NEW merged size: (19615, 11516)
dataframe size: (19615, 1116), NEW merged size: (19615, 12632)
dataframe size: (19615, 1469), NEW merged size: (19615, 14101)
dataframe size: (19615, 1063), NEW merged size: (19615, 15164)
dataframe size: (19615, 1668), NEW merged size: (19615, 16832)
dataframe size: (19615, 1421), NEW merged size: (19615, 18253)
dataframe size: (19615, 1358), NEW merged size: (19615, 19611)


In [26]:
merged.to_csv(os.path.join(output_dir, 'all_scRNA_barcodes_counts.conf{}.three_prime_utr.txt'.format(conf)), sep='\t')

In [27]:
print("wrote to: {}".format(os.path.join(output_dir, 'all_scRNA_barcodes_counts.conf{}.three_prime_utr.txt'.format(conf))))

wrote to: /home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/outputs/all_scRNA_barcodes_counts.conf0.9.three_prime_utr.txt


In [28]:
print("wrote to: {}".format(os.path.join(output_dir, 'all_scRNA_barcodes_counts.conf{}.txt'.format(conf))))

wrote to: /home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/outputs/all_scRNA_barcodes_counts.conf0.9.txt
